# Rasch (Nx1PL) MLTM .. subtractLayer
(Based on Cen 2009)
This model is used to explain the power law in learning.  In this notebook we try to build a neuralised version of the AFM and train it using simulated data.  The aim of using the AFM is to disentangle the latent traits that make up the overall score going into the sigmoid probability estimator.

The model is compensatory, which is a weakness.

In [1]:
from collections import defaultdict, Counter
from copy import copy
from math import exp, sqrt
from random import random, shuffle, choice, randint, uniform
import numpy

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.constraints import non_neg, max_norm
from numpy import array, mean, ones
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, multiply, subtract, add, Activation, Lambda, Flatten
from keras.layers import Dense, concatenate, MaxPooling1D, LocallyConnected1D, Reshape
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras import constraints
from keras.initializers import RandomNormal

from utils import generate_student_name
import random

from matplotlib import pyplot as plt

n_traits = 5



Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
generate_student_name()

'GAR GUY '

In [3]:
from keras import backend as K
from keras.constraints import Constraint
from keras.engine.topology import Layer
from keras import initializers

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, min_w=0, max_w=4):
        self.min_w = min_w
        self.max_w = max_w

    def __call__(self, p):
        return K.clip(p, self.min_w, self.max_w)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'min_w': self.min_w,
                'max_w': self.max_w }


class ProductLayer(Layer):
    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=-4.0, max_w=4.0), minv=-4,maxv=4, **kwargs):
        
        self.output_dim = output_dim
        super(ProductLayer, self).__init__(**kwargs)
        self.kernel_constraint= constraints.get(kernel_constraint)
        self.min_v = minv
        self.max_v = maxv

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, self.output_dim),
                                      initializer=initializers.RandomUniform(minval=self.min_v,maxval=self.max_v),
#                                       initializer=initializers.Constant(value=2.0),
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        
        super(ProductLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        z = K.concatenate([x,self.kernel], axis=0)
        p = K.prod(z, axis=0)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
    
class DifferenceLayer(Layer):
    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=-4.0, max_w=4.0), mu=5,sig=2.5, invert=False, **kwargs):
        
        self.output_dim = output_dim
        super(DifferenceLayer, self).__init__(**kwargs)
        self.kernel_constraint= constraints.get(kernel_constraint)
        self.mu = mu
        self.sig = sig
        self.invert = invert

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1,self.output_dim),
                                      initializer=RandomNormal(mean=self.mu, stddev=self.sig),
                                      #initializer=initializers.RandomUniform(),#minval=self.min_v,maxval=self.max_v),
#                                       initializer=initializers.Constant(value=2.0),
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        
        super(DifferenceLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
#         z = K.concatenate([x,-self.kernel], axis=2)
#         print("post K concat:", z.shape)

#         print("kernel", K.get_value(self.kernel))
#         print("xval", K.get_value(x))
        if self.invert:
            p = self.kernel - x
        else:
            p = x - self.kernel
        print("post K sum shaep:", p.shape)
#         print("pval", K.get_value(p))
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [4]:
for z in [-20, -10, -4,-3,-2,-1,0,1,2,3,4]:
    print( 1/(1+exp(-z)) )

2.0611536181902037e-09
4.5397868702434395e-05
0.01798620996209156
0.04742587317756678
0.11920292202211755
0.2689414213699951
0.5
0.7310585786300049
0.8807970779778823
0.9525741268224334
0.9820137900379085


In [5]:
class Question():
    def __init__(self, qix, nt=None, nnw=None, optimiser=None):
        #self.MAX_BETA = 15
        self.id = qix
        no_dummies = randint(1,(nt-1))
        print("no_dummies=",no_dummies)
        not_present=-20
        min_diff = 0
        max_diff = 10
        self.betas = [ not_present for _ in range(nt) ]
        print("nt=",nt)
        choices = [ x for x in range(nt) ]
        for _ in range(no_dummies):
            ch = random.choice(choices)
            choices.remove(ch)
            self.betas[ch]= round(numpy.random.normal(loc=5,scale=2.5))  #randint(min_diff,max_diff)
        #print("Made q with betas:", self.betas)
        
        th = Input(shape=(nnw,), name="user_theta")
#         b_in = Input(shape=(nnw,), name="beta_in")
        b = DifferenceLayer(nnw, kernel_constraint=WeightClip(min_w=not_present, max_w=max_diff), mu=5,sig=2.5) # introduce nt trainable weights for the beta values
#         b = DifferenceLayer(nnw, kernel_constraint=None) # introduce nt trainable weights for the beta values
#         b = bvar(b_in)
#         dif = subtract([th,b])
        dif = b(th)
        print("dif",dif.shape)
#         Pr = Dense(1, activation="sigmoid")(dif)
        Prs = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="qPr_sigmoids")(dif)
        print("Prs",Prs.shape)
        Pr = Lambda(lambda ps: K.prod(ps, axis=1), name="qPr_prod")(Prs)
        print("qPr_prod",Pr.shape)
        Pr = Reshape((1,))(Pr)
        print("qPr",Pr.shape)
        model = Model(inputs=th, outputs=Pr)
#         o = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#         optimiser = Adam(lr=0.1)
        model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])
        self.model = model
        self.pred_betas = b
        
        
# qs = [Question(qix, n_traits) for qix in range(10)]
# qs[0].model.summary()

# random.seed(666)
# qs = [Question(qix, n_traits) for qix in range(10)]
# for q in qs:
#     print(q.id, q.betas)

In [6]:
class Student():
    def __init__(self, nt=None, nnw=None, optimiser=None):
        #self.MAX_BETA = 15
        self.name = generate_student_name()
        min_abil = 5
        max_abil = 15
#         self.thetas = [ randint(min_abil, max_abil) for _ in range(nt) ]
        self.thetas = [ round(numpy.random.normal(loc=10, scale=2.5)) for _ in range(nt) ]
#         self.mastery = [0 for _ in range(nq)]
#         self.o_practice = [0 for _ in range(nq)]
#         self.h_practice = [0 for _ in range(nt)]
        #print("Made q with betas:", self.betas)
#         th_in = Input(shape=(nnw,), name="theta_in")
        b = Input(shape=(nnw,), name="qn_beta")
        th_var = DifferenceLayer(nnw, kernel_constraint=WeightClip(min_w=min_abil, max_w=max_abil), mu=10, sig=2.5, invert=True) # introduce nt trainable weights for the beta values
        dif = th_var(b)
#         dif = subtract([th,b])
        print("dif",dif.shape)
        Prs = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="sPr_sigmoids")(dif)
        print("sPrs",Prs.shape)
        Pr = Lambda(lambda ps: K.prod(ps, axis=1), name="sPr_prod")(Prs)
        print("sPr",Pr.shape)
        Pr = Reshape((1,))(Pr)
        model = Model(inputs=b, outputs=Pr)
#         o = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#         optimiser= Adam(lr=0.1)
        model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])
        self.model = model
        self.pred_theta = th_var

In [7]:
def attempt_q(student: Student, q: Question):
    p = calculate_pass_probability(student.thetas, q.betas)
    this_att = uniform(0,1)
    if (this_att <= p):
        passed=True
#         student.mastery[q.id] = 1
    else:
        passed=False

    return passed

In [8]:
def calculate_pass_probability(thetas, betas):
    # additive factors model is:
    # p_pass = 1 / 1 + exp(-z)
    # where z = a + sum[1:n]( -b + gT )
    
    p_pass = 1.0
    print("th,b",thetas,betas)
    for th,b in zip(thetas,betas):
        if b < 0:
            continue
        z = (th-b)
        p_pass_step = 1.0 / (1.0 + exp(-z))
        p_pass *= p_pass_step # simple conjunctive model of success!
    try:        
        print("p_pass={}".format(p_pass))
    except OverflowError:
        p_pass = 0.0
    #print("real p_pass = {}".format(p_pass))
    return p_pass
    

In [9]:
qopt = Adam(lr=0.1)

def create_qs(n_qs, nt=n_traits, nnw=n_traits, optimiser=qopt):
    random.seed(666)
    master_qs = [Question(qix, nt=nt, nnw=nnw, optimiser=optimiser) for qix in range(n_qs)]
    for q in master_qs:
        nocomps = len(q.betas)
        mag = sqrt(sum([ pow(b, 2) for b in q.betas if b!=-10 ]))
        print("Q:{}, difficulty={:.2f} across {} components".format(q.id, mag, nocomps))
    return master_qs

# Training
This is where sh!t gets real.  We take our tr_len (1000?) students, and iterate over them 100 times to create 100,000 *complete examples* of a student attacking the curriculum.  The questions themselves are attacked in random order: the student has no intelligent guidance throught the material. (Obvious we may wish to provide that guidance at some point in the future.)

Remember, there are only 12 exercises in the curriculum, so if the student is taking 60 or 70 goes to answer them all, that's pretty poor.  But some of these kids are dumb as lumber, so cut them some slack!  They will all get there in the end since by the CMU AFM practice will, eventually, make perfect!

In [10]:
# psi_opt = Adam(lr=0.1)
def create_students(n_students, nt=n_traits, nnw=n_traits, optimiser=None):
    random.seed(666)
    psi_list = [ Student(nt=nt, nnw=nnw, optimiser=optimiser) for _ in range(n_students)]
    for psi in psi_list:
        print(psi.name, psi.thetas)
    return psi_list
    

In [11]:
extend_pop=False
extend_by = 90
if extend_pop:
    for _ in range(extend_by):
        nu_psi = Student(nt=n_traits, nq=len(master_qs), optimiser=psi_opt)
        psi_list.append(nu_psi)

In [12]:
import gc
def generate_attempts(master_qs, psi_list):
    attempts_by_psi = {}
    attempts_by_q = {}

    user_budget = 100
    user_patience = 10
    for run in range(1):
        print("----{}\n".format(run))
        for psi in psi_list:
            spend=0
            #psi.mastery = [0 for _ in range(nq)]
            qs = [ix for ix in range(len(master_qs))]
            print("* * * **** USER {}".format(psi.name))
            print("* * * * ** THETAS {}".format(psi.thetas))

            while(True):
                q_ct = 0
                qix = random.choice(qs)
                q = master_qs[qix]
                passed=False

                if psi.name not in attempts_by_psi:
                    attempts_by_psi[psi.name]=[]

                if q not in attempts_by_q:
                    attempts_by_q[q]=[]

                while not passed and q_ct<user_patience:
                    passed = attempt_q(psi, q)
                    tup = (psi, q, passed)
                    #attempts.append(tup)
                    attempts_by_psi[psi.name].append(tup)
                    attempts_by_q[q].append(tup)
                    q_ct+=1

                if passed:
                    print("passed")
                    qs.remove(qix)

                spend += 1

                if qs == [] or spend>=user_budget:
                        print("* ** *QFIN USER {}".format(psi.name))
                        break
    gc.collect()
    return attempts_by_psi, attempts_by_q

In [13]:
import os
import IPython

def calibrate(master_qs, psi_list, attempts_by_q, attempts_by_psi, n_iter=20, record_param_fit=False):
    es = EarlyStopping(monitor="loss", mode="auto")
    random.seed(666)
    min_mse = 1000
    min_loss = 1000
    min_avg_fit_rmse = 1000
    mses=[]
    accs=[]
    q_outer_mses = []
    q_outer_accs = []
    s_outer_mses = []
    s_outer_accs = []
    th_mses = []
    b_mses = []
    th_accs= []
    b_accs =[]
    avg_fit_rmses = []
    th_fit_rmses = []
    b_fit_rmses = []
    losses = []
    patience_ct = 1
    max_pat = 0
    for ix in range(n_iter):
        q_clf_accs = []
        q_clf_mses = []
        q_clf_losses = []
        for q in master_qs:
            print(q.id)
            thetas = [ tup[0].pred_theta.get_weights()[0][0] for tup in attempts_by_q[q]]
            pfs = [ [tup[2]] for tup in attempts_by_q[q]]
            preb = q.pred_betas.get_weights()[0][0]
            
            arr_thetas = array(thetas)
            arr_pfs = array(pfs)
#             arr_ones = numpy.ones( (len(thetas),len(thetas[0])) )            
#             print("q arr_ones, shape:", arr_ones.shape)
            
#             prdz = q.model.predict(arr_thetas)
#             for pr,rr in zip(prdz, arr_pfs):
#                 print(pr,rr)
#             input("q predicts complete. continue...")
            
            h = q.model.fit(x=arr_thetas, y=arr_pfs, verbose=0, batch_size=1, epochs=100, callbacks=[es])
            del h
            
            loss, mse, acc = q.model.evaluate(x=arr_thetas, y=arr_pfs, verbose=0)
            q_clf_accs.append(acc)
            q_clf_mses.append(mse)
            q_clf_losses.append(loss)
            postb = q.pred_betas.get_weights()[0][0]
            print("q{} true({}), preb {} postb {}".format(q.id, q.betas, preb, postb))

        s_clf_accs=[]
        s_clf_mses=[]
        s_clf_losses = []
        for _student in psi_list:
            betas = [ tup[1].pred_betas.get_weights()[0][0] for tup in attempts_by_psi[_student.name]]# if tup[1]==q]
            pfs = [ [tup[2]] for tup in attempts_by_psi[_student.name]]# if tup[1]==q]
            preth= _student.pred_theta.get_weights()[0][0] 

            arr_betas = array(betas)
            arr_pfs = array(pfs)
#             arr_ones = numpy.ones( (len(betas),len(betas[0])) )
#             print("student shape arr ones = ", arr_ones.shape)
            
#             prdz = _student.model.predict(arr_thetas)
#             for pr,rr in zip(prdz, arr_pfs):
#                 print(pr,rr)
#             input("student predicts conmplete. continue...")

            h = _student.model.fit(x=arr_betas, y=arr_pfs, verbose=0, batch_size=1, epochs=100, callbacks=[es])
            del h
            
            loss, mse, acc = _student.model.evaluate(x=arr_betas, y=arr_pfs, verbose=0)
            s_clf_accs.append(acc)
            s_clf_mses.append(mse)
            s_clf_losses.append(loss)
            postth= _student.pred_theta.get_weights()[0][0]
            print("s{}: true({}), {} -> {}".format(_student.name, _student.thetas, preth, postth))

#         th_rmse = sqrt(mean([(numpy.sort(s.thetas) - numpy.sort(s.pred_theta.get_weights()[0][0]))**2 for s in psi_list]))
#         b_rmse = sqrt(mean([(numpy.sort(q.betas) - numpy.sort(q.pred_betas.get_weights()[0][0]))**2 for q in master_qs]))

#         avg_fit_rmse = (th_rmse + b_rmse)/2.0
        mse = (mean(s_clf_mses) + mean(q_clf_mses))/2.0
        acc = (mean(s_clf_accs) + mean(q_clf_accs)) /2.0
        loss = (mean(s_clf_losses) + mean(q_clf_losses)) /2.0
        
        if(record_param_fit):
            th_rmse = sqrt(mean([(numpy.sort(s.thetas) - numpy.sort(s.pred_theta.get_weights()[0][0]))**2 for s in psi_list]))
            b_rmse = sqrt(mean([(numpy.sort(q.betas) - numpy.sort(q.pred_betas.get_weights()[0][0]))**2 for q in master_qs]))

        print("iteration",ix)
#         print("b rmse:", b_rmse, "del:", (beta_rmses[-1] - b_rmse) if beta_rmses else 0)
#         print("th rmse: {} del: {}".format(th_rmse, (theta_rmses[-1] - th_rmse) if theta_rmses else 0))
        print("avg clf mse: {} del: {}".format(mse, (mses[-1] - mse) if mses else 0))
        print("avg clf loss {} del {}".format(loss, (losses[-1]-loss) if losses else 0))
        print("acc = {}".format(acc))
#         theta_rmses.append(th_rmse)
#         beta_rmses.append(b_rmse)
        mses.append(mse)
        accs.append(acc)

        if record_param_fit:
            th_fit_rmses.append(th_rmse)
            b_fit_rmses.append(b_rmse)
        
        th_mses.append(mean(s_clf_mses))
        b_mses.append(mean(q_clf_mses))
        th_accs.append(mean(s_clf_accs))
        b_accs.append(mean(q_clf_accs))
#         avg_fit_rmses.append(avg_fit_rmse)
#         s_outer_mses.append(mean(s_clf_mses))
#         s_outer_accs.append(mean(s_clf_accs))
#         q_outer_mses.append(mean(q_clf_mses))
#         q_outer_accs.append(mean(q_clf_accs))
        losses.append(loss)

#         if(min_fit_rmse > avg_fit_rmse):
#             min_fit_rmse = avg_fit_rmse
        
#         if(min_loss > loss):
#             min_loss = loss
#             patience_ct = 0
#         else:
#             patience_ct += 1
#         if patience_ct > max_pat:
#             print("patience exhausted:",max_pat,"iterations with no new clf MSE minimum")
#             break

        duration = .3  # second
        freq = 440  # Hz
        os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

        gc.collect()

#     print("final beta rmse = {}".format(beta_rmses[-1]))
#     print("final theta rmse = {}".format(theta_rmses[-1]))
#     print("final avg fit rmse = {}".format(avg_fit_rmses[-1]))
    print("final classifier mse = {}".format(mses[-1]))
    print("final acc = {}".format(accs[-1]))
    print("final loss = {}".format(losses[-1]))
    print("min mse =", min_mse)

    for m in [1, 1.5, 2, 2.5, 3]:
        duration = .1  # second
        freq = 440*m  # Hz
        os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))
#     jingle = IPython.display.Audio("calibration_complete.mp3", autoplay=True)
#     del jingle

    if record_param_fit:
        return th_fit_rmses,b_fit_rmses
    else:
        return (mses,accs, th_mses,th_accs, b_mses,b_accs)

In [ ]:
n_traits = 5
nn_dimensions = [5]
serieses = []
min_errs = []
n_qs = 10
n_students = 100
for ix,nnw in enumerate(nn_dimensions):
    qs = create_qs(n_qs, n_traits, nnw, optimiser=qopt)
    ss = create_students(n_students, n_traits, nnw, optimiser=qopt)
    attempts_by_psi, attempts_by_q = generate_attempts(qs,ss)
    print(qs[0].pred_betas.get_weights()[0][0])
    print(ss[0].pred_theta.get_weights()[0][0])
    th_rmses,b_rmses = calibrate(qs,ss,attempts_by_q, attempts_by_psi, n_iter=50, record_param_fit=True)
#     serieses += resultz
    serieses.append( (th_rmses,b_rmses) )
#     if ix < len(serieses): #append to old series
#         (_mses,_accs) = serieses[ix]
#         _mses += mses
#         _accs += accs
#         serieuses[ix] = (_mses,_accs)
#     else: #create new series
#         serieses.append((mses,accs))
    gc.collect()
from pygame import mixer
mixer.init()
mixer.music.load('calibration_complete.mp3')
mixer.music.play()

xs = range(len(th_rmses))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
for th_rmses, b_rmses in serieses:
    plt.plot(xs, numpy.multiply(1,th_rmses), label="w=5, students"),
    plt.plot(xs, numpy.multiply(1,b_rmses), label="w=5, qns")

# plt.plot(xs, accs, "m")
# plt.plot(xs, numpy.multiply(1,s_outer_accs), "g")
# plt.plot(xs, numpy.multiply(1,q_outer_accs), "k")

# plt.plot(xs, numpy.multiply(1,mses), "m--")
# plt.plot(xs, numpy.multiply(1,s_outer_mses), "g--")
# plt.plot(xs, numpy.multiply(1,q_outer_mses), "k--")
fig = plt.gcf()
fig.set_size_inches(8, 5)
plt.xlabel("#iterations")
plt.ylabel("fit error (RMSE)")
plt.suptitle("Neural-MLTM Parameter Fitting")
plt.title("(skills=5, items=10, students=100)")
plt.legend()
plt.show()


no_dummies= 4
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 3
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 4
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 3
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 2
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 4
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 3
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 1
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 1
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
no_dummies= 1
nt= 5
post K sum shaep: (?, 5)
dif (?, 5)
Prs (?, 5)
qPr_prod (?,)
qPr (?, 1)
Q:0, difficulty=24.04 across 5 components
Q:1, difficulty=29.58 across 5 compone

passed
th,b [9, 14, 13, 8, 13] [8, 8, -20, 5, 5]
p_pass=0.6944326271736855
passed
th,b [9, 14, 13, 8, 13] [1, 7, 5, -20, -20]
p_pass=0.9984189719441939
passed
* ** *QFIN USER NAS CHORAPH 
* * * **** USER SIN JAVIP 
* * * * ** THETAS [11, 14, 5, 7, 12]
th,b [11, 14, 5, 7, 12] [1, -20, 7, 6, 2]
p_pass=0.08713640658895327
th,b [11, 14, 5, 7, 12] [1, -20, 7, 6, 2]
p_pass=0.08713640658895327
th,b [11, 14, 5, 7, 12] [1, -20, 7, 6, 2]
p_pass=0.08713640658895327
th,b [11, 14, 5, 7, 12] [1, -20, 7, 6, 2]
p_pass=0.08713640658895327
th,b [11, 14, 5, 7, 12] [1, -20, 7, 6, 2]
p_pass=0.08713640658895327
th,b [11, 14, 5, 7, 12] [1, -20, 7, 6, 2]
p_pass=0.08713640658895327
passed
th,b [11, 14, 5, 7, 12] [3, -20, -20, -20, -20]
p_pass=0.9996646498695336
passed
th,b [11, 14, 5, 7, 12] [-20, -20, -20, -20, 1]
p_pass=0.999983298578152
passed
th,b [11, 14, 5, 7, 12] [3, -20, -20, 2, 3]
p_pass=0.9928515157826295
passed
th,b [11, 14, 5, 7, 12] [4, -1, 2, 3, -20]
p_pass=0.9345886939976814
passed
th,b [11, 14,

p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [5, 5, 11, 12, 9] [8, 8, -20, 5, 5]
p_pass=0.002206746329173431
th,b [

th,b [9, 11, 14, 11, 12] [-20, 2, -20, 5, -20]
p_pass=0.997404287375665
passed
th,b [9, 11, 14, 11, 12] [-20, -20, -20, -20, 1]
p_pass=0.999983298578152
passed
th,b [9, 11, 14, 11, 12] [4, -1, 2, 3, -20]
p_pass=0.9929679423877815
passed
th,b [9, 11, 14, 11, 12] [3, -20, -20, 2, 3]
p_pass=0.9972812130965374
passed
th,b [9, 11, 14, 11, 12] [1, 7, 5, -20, -20]
p_pass=0.9815633370461758
passed
th,b [9, 11, 14, 11, 12] [-20, 5, -20, -20, -20]
p_pass=0.9975273768433653
passed
* ** *QFIN USER PYCHO LYL 
* * * **** USER HIJA CHIPH 
* * * * ** THETAS [6, 13, 9, 12, 6]
th,b [6, 13, 9, 12, 6] [-20, -20, -20, -20, 1]
p_pass=0.9933071490757153
passed
th,b [6, 13, 9, 12, 6] [-20, 2, -20, 5, -20]
p_pass=0.9990722625996018
passed
th,b [6, 13, 9, 12, 6] [1, -20, 7, 6, 2]
p_pass=0.8570414693450917
passed
th,b [6, 13, 9, 12, 6] [4, -1, 2, 3, -20]
p_pass=0.8798860401841282
passed
th,b [6, 13, 9, 12, 6] [-20, 5, -20, -20, -20]
p_pass=0.9996646498695336
passed
th,b [6, 13, 9, 12, 6] [3, -20, -20, 2, 3]
p_pa

th,b [11, 7, 9, 4, 6] [4, -1, 2, 3, -20]
p_pass=0.7297271218365885
passed
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [1, -20, 7, 6, 2]
p_pass=0.10310046797776158
th,b [11, 7, 9, 4, 6] [-20, -20, -20, -20, 1]
p_pass=0.9933071490757153
passed
th,b [11, 7, 9, 4, 6] [8, 8, -20, 5, 5]
p_pass=0.05036935048788166
th,b [11, 7, 9, 4, 6] [8, 8, -20, 5, 5]
p_pass=0.05036935048788166
th,b [11, 7, 9, 4, 6] [8, 8, -20, 5, 5]
p_p

...

sVESU KUTHON : true([9, 4, 10, 10, 7]), [ 15.          15.           5.88213444  10.2083931   15.        ] -> [ 15.         15.          5.9157505  10.580369   15.       ]
sWIPIW FUCYJ : true([8, 13, 10, 12, 4]), [ 14.11872864   7.44176102  14.99983501   7.5024271   15.        ] -> [ 14.14860058   7.29539537  15.           7.61841679  15.        ]
sCHYFO BAB : true([10, 8, 12, 11, 16]), [ 15.          15.          14.05420589   7.58169079  15.        ] -> [ 15.          15.          14.14644241   7.87900448  15.        ]
sCYRIB SAKE : true([8, 10, 8, 11, 11]), [  9.57592487  15.          15.           8.3150177   15.        ] -> [  9.30261326  15.          15.           8.72504139  15.        ]
sNOFET THAHU : true([7, 11, 11, 10, 11]), [ 15.          15.          14.41408348   8.92248631  15.        ] -> [ 15.          15.          14.48587227   9.11730099  15.        ]
sLITHAS CEJ : true([9, 11, 9, 9, 13]), [ 15.           8.22457314  15.          15.          15.        ] -> [ 15.   

sLASAPH LICI : true([11, 11, 11, 5, 7]), [ 15.         15.         15.         15.          5.0079422] -> [ 15.         15.         15.         15.          5.0207243]
sPHIRO SYDOP : true([6, 9, 11, 10, 8]), [ 14.55888367  15.          15.           9.5479126   15.        ] -> [ 14.59328556  15.          15.           9.82766151  15.        ]
sMIWYPH GEL : true([11, 7, 9, 9, 8]), [  9.3985548    8.19563007  12.98666763   8.83340836  15.        ] -> [  9.18531322   8.19520092  13.02712059   8.48156548  15.        ]
sFOK SIWI : true([10, 10, 13, 10, 9]), [ 15.  15.  15.  15.  15.] -> [ 15.  15.  15.  15.  15.]
sNIBI WON : true([10, 11, 11, 10, 9]), [ 15.  15.  15.  15.  15.] -> [ 15.  15.  15.  15.  15.]
sLUR BIH : true([9, 10, 13, 11, 6]), [ 15.  15.  15.  15.  15.] -> [ 15.  15.  15.  15.  15.]
sNAS CHORAPH : true([9, 14, 13, 8, 13]), [  9.33194733  15.          15.          15.          15.        ] -> [  9.23547935  15.          15.          15.          15.        ]
sSIN JAVIP : tru

sYUK FYKO : true([6, 6, 17, 16, 15]), [ 15.          15.           8.27838135   7.16354942  15.        ] -> [ 15.          15.           8.53889275   7.54692698  15.        ]
sVESU KUTHON : true([9, 4, 10, 10, 7]), [ 15.         15.          5.9157505  10.580369   15.       ] -> [ 15.          15.           6.04639149  10.78950977  15.        ]
sWIPIW FUCYJ : true([8, 13, 10, 12, 4]), [ 14.14860058   7.29539537  15.           7.61841679  15.        ] -> [ 14.17987251   7.55506229  15.           7.79741669  15.        ]
sCHYFO BAB : true([10, 8, 12, 11, 16]), [ 15.          15.          14.14644241   7.87900448  15.        ] -> [ 15.          15.          14.20571327   7.82615328  15.        ]
sCYRIB SAKE : true([8, 10, 8, 11, 11]), [  9.30261326  15.          15.           8.72504139  15.        ] -> [  9.25840282  15.          15.           8.70744038  15.        ]
sNOFET THAHU : true([7, 11, 11, 10, 11]), [ 15.          15.          14.48587227   9.11730099  15.        ] -> [ 15.    

sLASAPH LICI : true([11, 11, 11, 5, 7]), [ 15.         15.         15.         15.          5.0207243] -> [ 15.          15.          15.          15.           5.03323984]
sPHIRO SYDOP : true([6, 9, 11, 10, 8]), [ 14.59328556  15.          15.           9.82766151  15.        ] -> [ 14.61027145  15.          14.96036053   9.41547394  15.        ]
sMIWYPH GEL : true([11, 7, 9, 9, 8]), [  9.18531322   8.19520092  13.02712059   8.48156548  15.        ] -> [  9.05844498   8.51037407  13.08093643   8.26627159  15.        ]
sFOK SIWI : true([10, 10, 13, 10, 9]), [ 15.  15.  15.  15.  15.] -> [ 15.  15.  15.  15.  15.]
sNIBI WON : true([10, 11, 11, 10, 9]), [ 15.  15.  15.  15.  15.] -> [ 15.  15.  15.  15.  15.]
sLUR BIH : true([9, 10, 13, 11, 6]), [ 15.  15.  15.  15.  15.] -> [ 15.  15.  15.  15.  15.]
sNAS CHORAPH : true([9, 14, 13, 8, 13]), [  9.23547935  15.          15.          15.          15.        ] -> [  9.31331921  15.          15.          15.          15.        ]
sSIN JAVIP 

sCEDU CEF : true([8, 15, 8, 8, 10]), [ 15.          15.          15.           8.12838554  15.        ] -> [ 15.          15.          14.99220943   8.15042973  15.        ]
sYUK FYKO : true([6, 6, 17, 16, 15]), [ 15.          15.           8.53889275   7.54692698  15.        ] -> [ 15.          15.           8.30886555   7.1211729   15.        ]
sVESU KUTHON : true([9, 4, 10, 10, 7]), [ 15.          15.           6.04639149  10.78950977  15.        ] -> [ 15.          14.99991798   5.49608374  10.35153198  15.        ]
sWIPIW FUCYJ : true([8, 13, 10, 12, 4]), [ 14.17987251   7.55506229  15.           7.79741669  15.        ] -> [ 14.21047401   7.29114437  15.           7.37479782  15.        ]
sCHYFO BAB : true([10, 8, 12, 11, 16]), [ 15.          15.          14.20571327   7.82615328  15.        ] -> [ 15.          15.          14.24822903   7.26233053  15.        ]
sCYRIB SAKE : true([8, 10, 8, 11, 11]), [  9.25840282  15.          15.           8.70744038  15.        ] -> [  8.9405

In [ ]:
n_traits = 5
nn_dimensions = [1,3,5,7,9,20]
serieses = []
min_errs = []
n_qs = 10
n_students = 100
for ix,nnw in enumerate(nn_dimensions):
    qs = create_qs(n_qs, n_traits, nnw, optimiser=qopt)
    ss = create_students(n_students, n_traits, nnw, optimiser=qopt)
    attempts_by_psi, attempts_by_q = generate_attempts(qs,ss)
    print(qs[0].pred_betas.get_weights()[0][0])
    print(ss[0].pred_theta.get_weights()[0][0])
    resultz = calibrate(qs,ss,attempts_by_q, attempts_by_psi, n_iter=20)
#     serieses += resultz
    serieses.append( resultz )
#     if ix < len(serieses): #append to old series
#         (_mses,_accs) = serieses[ix]
#         _mses += mses
#         _accs += accs
#         serieuses[ix] = (_mses,_accs)
#     else: #create new series
#         serieses.append((mses,accs))
    gc.collect()
from pygame import mixer
mixer.init()
mixer.music.load('calibration_complete.mp3')
mixer.music.play()

In [ ]:
# import pickle
# pickle.dump( serieses, open( "serieses.p", "wb" ) )

xmax=10
xs = range(len(serieses[0][0]))[0:xmax]
print(len(serieses))
for s in serieses:
    print(len(s))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
fig, axes = plt.subplots(nrows=2, ncols=2)
print(axes.shape)
xs = range(xmax)
for (mses,accs, th_mses,th_accs, b_mses,b_accs),c,d in zip(serieses,["r--","y--","k-","c--","b--","m--"],nn_dimensions):
#     axes[0,0].plot(range(min(xmax,len(mses))), numpy.multiply(1,mses[0:xmax]), c, label="nnw={}".format(d))
#     axes[0,1].plot(range(min(xmax,len(accs))), numpy.multiply(1,accs[0:xmax]), c, label="nnw={}".format(d))
    axes[0,0].plot(range(min(xmax,len(th_mses))), numpy.sqrt(th_mses[0:xmax]), c, label="nnw={}".format(d))
    axes[0,1].plot(range(min(xmax,len(th_accs))), numpy.sqrt(th_accs[0:xmax]), c, label="nnw={}".format(d))
    axes[1,0].plot(range(min(xmax,len(b_mses))), numpy.sqrt(b_mses[0:xmax]), c, label="nnw={}".format(d))
    axes[1,1].plot(range(min(xmax,len(b_accs))), numpy.sqrt(b_accs[0:xmax]), c, label="nnw={}".format(d))

# plt.plot(xs, accs, "m")
# plt.plot(xs, numpy.multiply(1,s_outer_accs), "g")7
# plt.plot(xs, numpy.multiply(1,q_outer_accs), "k")

# plt.plot(xs, numpy.multiply(1,mses), "m--")
# plt.plot(xs, numpy.multiply(1,s_outer_mses), "g--")
# plt.plot(xs, numpy.multiply(1,q_outer_mses), "k--")
for ix in range(axes.shape[0]):
    subcats = ["Student","Question"]
    for iy in range(axes.shape[1]):
        axes[ix,iy].set_xlabel("#iterations")
        axes[ix,iy].legend()
        if iy==0:
            axes[ix,iy].set_title("{} fit error".format(subcats[ix]))
            axes[ix,iy].set_ylabel("fit error (RMSE)")
        else:
            axes[ix,iy].set_title("{} fit accuracy".format(subcats[ix]))
            axes[ix,iy].set_ylabel("prediction accuracy")

fig.suptitle("Neural-MLTM Parameter Fitting (k={}, q={}, s={})".format(n_traits, n_qs, n_students))
fig.set_size_inches(18, 18)


fig.show()

In [ ]:
xs = range(len(serieses[0][0]))
print(len(serieses))
for s in serieses:
    print(len(s))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
fig, axes = plt.subplots(nrows=1, ncols=2)
print(axes.shape)

min_mses = []
max_accs = []
bmin_mses = []
bmax_accs = []

for (mses,accs, th_mses,th_accs, b_mses,b_accs),d in zip(serieses,nn_dimensions):
    mm = min(numpy.sqrt(th_mses))
    ma = max(numpy.sqrt(th_accs))
    min_mses.append(mm)
    max_accs.append(ma)
    mm = min(numpy.sqrt(b_mses))
    ma = max(numpy.sqrt(b_accs))
    bmin_mses.append(mm)
    bmax_accs.append(ma)
    

axes[0].plot(nn_dimensions, numpy.multiply(1,min_mses))
axes[0].plot(nn_dimensions, numpy.multiply(1,bmin_mses))
axes[1].plot(nn_dimensions, numpy.multiply(1,max_accs))
axes[1].plot(nn_dimensions, numpy.multiply(1,bmax_accs))
axes[0].axvline(x=5, linestyle="--")
axes[1].axvline(x=5, linestyle="--")
axes[0].set_xticks(nn_dimensions)
axes[1].set_xticks(nn_dimensions)

fig.suptitle("Neural-MLTM Parameter Fitting (k={}, q={}, s={})".format(n_traits, n_qs, n_students))
fig.set_size_inches(18,4)

fig.show()